In [1]:
from src.dataset.feedback_utils_v2 import Feedback
from src.dataset.format_v2 import to_dpo, to_sft, to_full, to_distill_sft
import json

feedback = Feedback(content = "Do not talk about elephant")
# sft_dataset = to_sft(feedback)
dataset = to_distill_sft(feedback)

Loaded 201 prompts
Loaded 201 search infos


In [4]:
feedback

In [2]:
from huggingface_hub import login
from os import getenv
# from google.colab import userdata
HF_TOKEN = getenv("HF_TOKEN")
login(
  token=HF_TOKEN, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# Getting rid of redundancy is the most important approach towards making progress
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "01-ai/Yi-1.5-9B-Chat"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer | Will be some funny error (mismatch parameters etc.) Rerun the kernel fixes it (how to do it more systematically)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings


if "Meta-Llama-3-" in model_id:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
else:
    tokenizer.pad_token = tokenizer.unk_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
model_id = "01-ai/Yi-1.5-9B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [4]:
from src.custom_collator import DataCollatorForCompletionOnlyLM_v2, get_format_func

# Ok at least colab's result is reproducible -- now continue
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)


args = TrainingArguments(
    output_dir="alignment-adaptor-test02", # directory to save and repository id
    num_train_epochs=10,                     # number of training epochs
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=False,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    remove_unused_columns=False,
)


In [83]:
from src.custom_collator import DataCollatorForCompletionOnlyLM_v2, get_format_func, get_teacher_format_func
from src.dataset.prompts_v2 import TEACHER_QUERY_TEMPLATE

messages = [
    {"role": "user",
     "content": "hi"},
    {"role": "assistant",
     "content": "hello"}
]
format_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print(f"Formatted Prompt from {model_id}: ")
print(format_prompt)

# Patterns are required to get the Teacher Query
template_patterns = {
    "user_start": "<|im_start|>user\n",
    "assistant_start": "<|im_start|>assistant\n",
    "end": "<im_end>"
}

response_template = "\n<|im_start|>assistant\n"
collator = DataCollatorForCompletionOnlyLM_v2(response_template, tokenizer=tokenizer)
formatting_prompt_func = get_format_func(tokenizer)
teacher_formatting_prompt_func = get_teacher_format_func(tokenizer)

get_teacher_query = lambda prompt, completion: TEACHER_QUERY_TEMPLATE.format(content = feedback.content, prompt=prompt, completion=completion)

Formatted Prompt from 01-ai/Yi-1.5-9B-Chat: 
<|im_start|>user
hi<|im_end|>
<|im_start|>assistant
hello<|im_end|>



In [84]:
from trl import SFTTrainer
# from src.icdft import InContextDistillTrainer
from src.dft import DFTTrainer

max_seq_length = 1024 # max sequence length for model and packing of the dataset

# This Works (!)
# trainer = SFTTrainer(
#     model=model,
#     args=args,
#     train_dataset=dataset["train"],
#     peft_config=peft_config,
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     # dataset_text_field="text", # Question: I do NOT think 'text' is one of the key in the dataset ??
#     # formatting_func=formatting_prompt_func,
#     data_collator=collator,
#     packing=False,
#     dataset_kwargs={
#         "add_special_tokens": False,  # We template with special tokens
#         "append_concat_token": False, # No need to add additional separator token
#     }
# )

trainer = DFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=formatting_prompt_func,
    teacher_formatting_func=teacher_formatting_prompt_func,
    data_collator=collator,
    response_template = response_template,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

# trainer = InContextDistillTrainer(
#     model=model,
#     args=args,
#     get_teacher_query=get_teacher_query,
#     template_patterns = template_patterns,
#     response_template=response_template,
#     train_dataset=dataset["train"],
#     peft_config=peft_config,
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     formatting_func=formatting_prompt_func,
#     data_collator=collator,
#     packing=False,
#     dataset_kwargs={
#         "add_special_tokens": False,  # We template with special tokens
#         "append_concat_token": False, # No need to add additional separator token
#     }
# )


# Test DFT instead

NameError: name 'model' is not defined

In [ ]:
# Test Trainer Object: Most importanly, the loss function should be working on a Batch

# Non Packed Dataloader
trainer._prepare_non_packed_dataloader(tokenizer, dataset["train"], max_seq_length=1024, formatting_func=formatting_prompt_func,
                                       add_special_tokens=False, remove_unused_columns=False)
# Loss Computation : Single Data Point
inputs = dataset["train"][0]
trainer.compute_loss(model, inputs)

# Loss Computation: Batched Data Points
inputs = dataset["train"][:2]
trainer.compute_loss(model, inputs)

# Debugging Batched Data Points Loss Computation
inputs = dataset["train"][:2]

from src.dft import convert_to_tensor, masked_self_dl

student_inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"],
    "labels": inputs["labels"],
}

teacher_inputs = {
    "input_ids": inputs["teacher_input_ids"],
    "attention_mask": inputs["teacher_attention_mask"],
    "labels": inputs["teacher_labels"],
}
# Convert to Tensor
student_batch = {
    key: convert_to_tensor(value) for key, value in student_inputs.items()
}
teacher_batch = {
    key: convert_to_tensor(value) for key, value in teacher_inputs.items()
}

loss, metric = trainer.compute_distillation_loss(model, student_batch, teacher_batch, train_eval="train")

with torch.cuda.amp.autocast():
    student_output = model(**student_batch)   

with torch.no_grad():
    with torch.cuda.amp.autocast():
        teacher_output = model(**teacher_batch)              
    
attention_mask = student_batch.get('attention_mask', None)
attention_mask_student = attention_mask * (student_batch["labels"] != -100)
teacher_logits = teacher_output.logits / trainer.kd_temperature
student_logits = student_output.logits / trainer.kd_temperature

# Compute Distillation Loss
self_distill_loss = masked_self_dl(student_logits, teacher_logits, attention_mask_student, trainer.use_avg_kl) * (self.kd_temperature ** 2)



In [ ]:
trainer.train()